## Aspect Based Sentiment Analysis: Routers


### This is a Natural Language Processing based solution which can detect up to 9 aspects and their respective sentiments from online product reviews for Routers.

This sample notebook shows you how to deploy Aspect Based Sentiment Analysis: Routers using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Aspect Based Sentiment Analysis: Routers. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page Product Review Aspect Based Sentiment Analysis: Routers. 
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn='arn:aws:sagemaker:us-east-2:786796469737:model-package/router-absa'

In [2]:
import base64
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from urllib.parse import urlparse
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import urllib.request
import numpy as np

In [3]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name='router-absa'

content_type='text/plain'

real_time_inference_instance_type='ml.m5.large'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [5]:

def predict_wrapper(endpoint, session):
    return sage.predictor.Predictor(endpoint, session,content_type)

#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

#Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

------!

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

In [6]:
file_name = 'input/sample.txt'

<Add code snippet that shows the payload contents>

#### C. Perform real-time inference

In [9]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $model_name \
    --body fileb://$file_name \
    --content-type $content_type \
    --region $sagemaker_session.boto_region_name \
    output.txt

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Visualize output

In [10]:
import json
with open('output.txt', 'r') as f:
    output = json.load(f)
print(json.dumps(output, indent = 1))

{
 "review": "Installation was pretty straight forward. What is nice about the way this dishwasher is configured is that the hot water connection can be accessed from the front of the machine if you remove the kick plate. So if I had to retighten the water connection or something I wouldn't have to drag the machine out from under the counter.\r\nThe instructions could have been written clearer, especially regarding the leveling feet. Heads up to first time dishwasher installers: It only comes with a drain hose. You have to supply the water connections/adapters as well as any appliance electrical plug (If you are not hard wiring it).\r\n\r\nIt's quiet. It cleans and dries very well. It looks very nice too. However, the baskets are designed for Western dishes and cookware and some Indian dishes, such as stainless steel plates with high sides don't fit easily in the baskets.",
 "topics": [
  {
   "aspect": {
    "Easy to Install": 0.8775739832058218
   },
   "sentence": "instal pretti str

#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [11]:
predictor=sage.predictor.Predictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [12]:
#upload the batch-transform job input files to S3
transform_input_folder = "input"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-786796469737/router-absa


In [13]:

#Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

..........................2022-02-04T11:37:09.553:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.20.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
2022-02-04 11:37:06.276753: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-04 11:37:06.277145: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-04 11:37:06.277378: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (506f9bd7017f): /proc/driver/nvidia/version does not exist
2022-02-04 11:37:06.277858: I tensorflow/core/platform/cp

169.254.255.130 - - [04/Feb/2022 11:37:33] "POST /invocations HTTP/1.1" 200 -


In [16]:
import os
s3_conn = boto3.client("s3")
with open('output2.txt', 'wb') as f:
    s3_conn.download_fileobj(bucket, os.path.basename(transformer.output_path)+'/sample.txt.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


In [17]:
with open('output2.txt', 'r') as f:
    output = json.load(f)
print(json.dumps(output, indent = 1))

{
 "review": "Installation was pretty straight forward. What is nice about the way this dishwasher is configured is that the hot water connection can be accessed from the front of the machine if you remove the kick plate. So if I had to retighten the water connection or something I wouldn't have to drag the machine out from under the counter.\r\nThe instructions could have been written clearer, especially regarding the leveling feet. Heads up to first time dishwasher installers: It only comes with a drain hose. You have to supply the water connections/adapters as well as any appliance electrical plug (If you are not hard wiring it).\r\n\r\nIt's quiet. It cleans and dries very well. It looks very nice too. However, the baskets are designed for Western dishes and cookware and some Indian dishes, such as stainless steel plates with high sides don't fit easily in the baskets.",
 "topics": [
  {
   "aspect": {
    "Easy to Install": 0.8775739832058218
   },
   "sentence": "instal pretti str

### 4. Clean-up

#### A. Delete the model

In [18]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

